In [1]:
# %matplotlib notebook

In [9]:
%matplotlib qt

In [24]:
from __future__ import annotations

import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize

from maneuvering.types import Scalar
from maneuvering.orbit.keplerian import Kep, KepTrue
from maneuvering.orbit.distance import distance_orbit
from maneuvering.maneuvers.quasi_circular.reference_orbit import trans_devs, reference_orbit
from maneuvering.maneuvers.quasi_circular.transition.noncoplanar.noncoplanar import noncoplanar_analytical
from maneuvering.maneuvers.quasi_circular.transition.analytical_transition import analytical_transition
from maneuvering.maneuvers.quasi_circular.transition.execute import execute, execute_batch
from maneuvering.utils.math_tools import normalize_angle
from maneuvering.maneuvers.maneuver import Maneuver

In [4]:
deg = math.pi / 180.0
mu = 3.9860044158e14

### Поверхности ошибок

In [10]:
def build_surfaces_for_di_draan(
    di_vals_deg: np.ndarray,
    draan_vals_deg: np.ndarray,
    *,
    a0: float = 6_800_000.0,
    e0: float = 0.0,
    w0_deg: float = 0.0,
    i0_deg: float = 30.0,
    raan0_deg: float = 30.0,
    mu: float = 3.986004418e14,
) -> tuple[np.ndarray, np.ndarray]:
    """
    Для сетки Δi и ΔΩ (в градусах) строит поверхности:
      init_err[j,i]  = distance(oi, ot)
      final_err[j,i] = distance(execute(oi, mans, μ), ot)
    где:
      oi = {a0, e0, w0,  i0,  Ω0}
      ot = {a0, e0, w0,  i0+Δi,  Ω0+ΔΩ}
    Индексы: i — ΔΩ, j — Δi.
    """
    w0 = math.radians(w0_deg)
    i0 = math.radians(i0_deg)
    raan0 = math.radians(raan0_deg)

    init_err = np.zeros((di_vals_deg.size, draan_vals_deg.size), dtype=np.float64)
    final_err = np.zeros_like(init_err)

    for j, di_deg in enumerate(di_vals_deg):
        for i, dO_deg in enumerate(draan_vals_deg):
            oi = Kep(a=a0, e=e0, w=w0, i=i0, raan=raan0)
            ot = Kep(
                a=a0,
                e=e0,
                w=w0,
                i=i0 + math.radians(float(di_deg)),
                raan=raan0 + math.radians(float(dO_deg)),
            )

            d0 = distance_orbit(oi, ot)

            mans = noncoplanar_analytical(oi, ot, mu)

            of_true = execute(
                oi=KepTrue(a=oi.a, e=oi.e, w=oi.w, i=oi.i, raan=oi.raan, nu=0.0),
                maneuvers=mans,
                mu=mu,
            )
            of_kep = Kep(a=of_true.a, e=of_true.e, w=of_true.w, i=of_true.i, raan=of_true.raan)
            d1 = distance_orbit(of_kep, ot)

            init_err[j, i] = d0
            final_err[j, i] = d1

    return init_err, final_err

In [12]:
# Диапазоны Δi и ΔΩ: [0°, 5°]
di_vals = np.linspace(0.0, 5.0, 41)     # Δi, deg
dO_vals = np.linspace(0.0, 5.0, 41)     # ΔΩ, deg
dI_grid, dO_grid = np.meshgrid(di_vals, dO_vals, indexing="ij")

init_err, final_err = build_surfaces_for_di_draan(di_vals, dO_vals)

# Одна фигура, один 3D-axes, две поверхности
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection="3d")
ax.set_title("Некомпланарный переход: ошибки vs Δi, ΔΩ")

# Начальная и финальная поверхности
# ax.plot_surface(dO_grid, dI_grid, init_err, rstride=1, cstride=1, linewidth=0, antialiased=True, alpha=0.95)
ax.plot_surface(dO_grid, dI_grid, final_err, rstride=1, cstride=1, linewidth=0, antialiased=True, alpha=0.6)

ax.set_xlabel("ΔΩ, [deg]")
ax.set_ylabel("Δi, [deg]")
ax.set_zlabel("distance, [m]", labelpad=6)

fig.tight_layout()
plt.show()

### Стоимость изменения различных элементов орбиты

In [14]:
def sum_delta_v(maneuvers) -> float:
    """Сумма модулей импульсов (характеристическая скорость), [м/с]."""
    return float(sum(np.linalg.norm(m.dv) for m in maneuvers))

In [15]:
def dv_vs_delta(
    oi: Kep,
    mu: Scalar,
    param: str,
    deltas: np.ndarray,
    *,
    degrees: bool = False,
) -> tuple[np.ndarray, np.ndarray]:
    """
    Строит зависимость ΣΔV(Δparam) при фиксированных остальных элементах.
    param ∈ {"a","e","w","i","raan"}; deltas — массив приращений соответствующей размерности.
    """
    x = deltas.copy()
    y = np.zeros_like(x, dtype=float)

    for k, d in enumerate(deltas):
        if degrees:
            d_val = math.radians(float(d))
        else:
            d_val = float(d)

        if param == "a":
            ot = Kep(a=oi.a + d_val, e=oi.e, w=oi.w, i=oi.i, raan=oi.raan)
        elif param == "e":
            ot = Kep(a=oi.a, e=oi.e + d_val, w=oi.w, i=oi.i, raan=oi.raan)
        elif param == "w":
            ot = Kep(a=oi.a, e=oi.e, w=oi.w + d_val, i=oi.i, raan=oi.raan)
        elif param == "i":
            ot = Kep(a=oi.a, e=oi.e, w=oi.w, i=oi.i + d_val, raan=oi.raan)
        elif param == "raan":
            ot = Kep(a=oi.a, e=oi.e, w=oi.w, i=oi.i, raan=oi.raan + d_val)
        else:
            raise ValueError(f"Unsupported param: {param}")

        mans = analytical_transition(oi, ot, mu)
        y[k] = sum_delta_v(mans)

    return x, y

In [32]:
# Базовые значения
a0 = 6_800_000.0
e0 = 0.001
w0 = math.radians(30.0)
i0 = math.radians(30.0)
raan0 = math.radians(30.0)
mu = 3.986004418e14

oi = Kep(a=a0, e=e0, w=w0, i=i0, raan=raan0)

# Диапазоны Δ (по ТЗ)
da = np.linspace(0.0, 500_000.0, 201)      # м
de = np.linspace(0.0, 0.05, 201)           # 1
dw = np.linspace(0.0, 90.0, 181)           # град
di = np.linspace(0.0, 5.0, 101)            # град
dO = np.linspace(0.0, 5.0, 101)            # град

# Вычисления
xa, ya = dv_vs_delta(oi, mu, "a", da, degrees=False)
xe, ye = dv_vs_delta(oi, mu, "e", de, degrees=False)
xw, yw = dv_vs_delta(oi, mu, "w", dw, degrees=True)
xi, yi = dv_vs_delta(oi, mu, "i", di, degrees=True)
xO, yO = dv_vs_delta(oi, mu, "raan", dO, degrees=True)

# Графики: 2×3, последний из нижнего ряда оставим пустым
fig, axes = plt.subplots(2, 3, figsize=(14, 8), sharey=False)
axes = axes.ravel()

tick_params_font_size = 18
label_font_size = 22
# a
axes[0].plot(xa, ya, lw=2)
axes[0].set_title("ΣΔV vs Δa", fontsize=label_font_size)
axes[0].set_xlabel("Δa, m", fontsize=label_font_size)
axes[0].set_ylabel("ΣΔV, m/s", fontsize=label_font_size)
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='both', which='major', labelsize=tick_params_font_size)

# e
axes[1].plot(xe, ye, lw=2)
axes[1].set_title("ΣΔV vs Δe", fontsize=label_font_size)
axes[1].set_xlabel("Δe, [1]", fontsize=label_font_size)
axes[1].grid(True, alpha=0.3)
axes[1].tick_params(axis='both', which='major', labelsize=tick_params_font_size)

# w
axes[2].plot(xw, yw, lw=2)
axes[2].set_title("ΣΔV vs Δω", fontsize=label_font_size)
axes[2].set_xlabel("Δω, deg", fontsize=label_font_size)
axes[2].grid(True, alpha=0.3)
axes[2].tick_params(axis='both', which='major', labelsize=tick_params_font_size)

# i
axes[3].plot(xi, yi, lw=2)
axes[3].set_title("ΣΔV vs Δi", fontsize=label_font_size)
axes[3].set_xlabel("Δi, deg", fontsize=label_font_size)
axes[3].set_ylabel("ΣΔV, m/s", fontsize=label_font_size)
axes[3].grid(True, alpha=0.3)
axes[3].tick_params(axis='both', which='major', labelsize=tick_params_font_size)

# raan
axes[4].plot(xO, yO, lw=2)
axes[4].set_title("ΣΔV vs ΔΩ", fontsize=label_font_size)
axes[4].set_xlabel("ΔΩ, deg", fontsize=label_font_size)
axes[4].grid(True, alpha=0.3)
axes[4].tick_params(axis='both', which='major', labelsize=tick_params_font_size)

# последний пустой
axes[5].axis("off")

fig.suptitle("Суммарная характеристическая скорость для вариаций отдельных элементов", y=0.98)
fig.tight_layout()
plt.show()

### Зависимости орбитальных элементов от абсолютного угла, пройденного КА, с активным маневрированием

In [33]:
def plot_transition_series(oi, ot, mu):
    """
    1) Считает манёвры: mans = analytical_transition(oi, ot, mu)
    2) Прогоняет: states, angles_abs = execute_batch(oi, mans, step, mu)
       (step фиксируем внутри как 2π/720)
    3) Строит 6 графиков (2×3): a, e, ω; i, Ω, ν.
       Ось X — накопленный абсолютный угол u=ω+ν (радианы).
    Возвращает: fig, axes, states, angles_abs
    """
    # 1) Манёвры
    mans = analytical_transition(oi, ot, mu)

    # 2) Прогон (фиксированный шаг по u)
    states, angles_abs = execute_batch(oi, mans, mu)

    if not states:
        raise ValueError("Пустой список состояний.")

    # 3) Достаём ряды
    a = np.array([s.a for s in states], dtype=float)
    e = np.array([s.e for s in states], dtype=float)
    w = np.array([s.w for s in states], dtype=float)
    inc = np.array([s.i for s in states], dtype=float)
    Om = np.array([s.raan for s in states], dtype=float)
    nu = np.array([s.nu for s in states], dtype=float)
    x = np.asarray(angles_abs, dtype=float)  # уже в радианах

    # Разворачиваем углы (чтобы не было скачков через 2π)
    unwrap = lambda arr: np.unwrap(arr, discont=np.pi)
    w_u, i_u, Om_u, nu_u = map(unwrap, (w, inc, Om, nu))

    # Оформление
    plt.rcParams.update({
        "figure.constrained_layout.use": True,
        "axes.grid": True,
        "font.size": 11,
        "figure.figsize": (12, 6),
    })

    fig, axes = plt.subplots(2, 3, sharex=True)
    (ax_a, ax_e, ax_w), (ax_i, ax_Om, ax_nu) = axes

    # Верхний ряд
    ax_a.plot(x, a, lw=2)
    ax_a.set_title("Большая полуось a, [м]")

    ax_e.plot(x, e, lw=2)
    ax_e.set_title("Эксцентриситет e, [1]")

    ax_w.plot(x, w_u, lw=2)
    ax_w.set_title("Аргумент перицентра ω, [рад]")

    # Нижний ряд
    ax_i.plot(x, i_u, lw=2)
    ax_i.set_title("Наклонение i, [рад]")

    ax_Om.plot(x, Om_u, lw=2)
    ax_Om.set_title("Долгота восходящего узла Ω, [рад]")

    ax_nu.plot(x, nu_u, lw=2)
    ax_nu.set_title("Истинная аномалия ν, [рад]")

    # Общая ось X
    axes[1, 1].set_xlabel("Накопленный угол u = ω + ν, [рад]")

    for ax in axes.ravel():
        ax.tick_params(axis='both', which='major', labelsize=10)

    plt.show()

In [35]:
# Некомпланарный, узловой, ΔE_x≠0
oi = KepTrue(a=7_000_000.0, e=0.00228, i=10 * deg, w=20 * deg, raan=130 * deg, nu=0.0)
ot = Kep(a=7_010_000.0, e=0.00149, i=15 * deg, w=150 * deg, raan=130 * deg)
plot_transition_series(oi, ot, mu)

In [36]:
# Некомпланарный, невырожденный, ΔE≠0
oi = KepTrue(a=7_000_000.0, e=0.00228, i=10 * deg, w=20 * deg, raan=130 * deg, nu=0.0)
ot = Kep(a=8_000_000.0, e=0.00149, i=15 * deg, w=150 * deg, raan=130 * deg)
plot_transition_series(oi, ot, mu)

In [37]:
# Некомпланарный, особый случай
oi = KepTrue(a=7_000_000.0, e=0.00228, i=10 * deg, w=90 * deg, raan=130 * deg, nu=0.0)
ot = Kep(a=7_010_000.0, e=0.09149, i=12 * deg, w=90 * deg, raan=130 * deg)
plot_transition_series(oi, ot, mu)